***Health Care Cost Minimization***

Andrea Figueroa, Kai Tsuyoshi

**Introduction** 

In those last years, the world has been through a financial crisis. Moreover, the uncertainty puts pressure on companies to reduce and optimize their costs. Beyond challenges in Health Care after Covid-19 there are financial challenges to providers. Shutting down hospitals, physician offices decreased capacity to provide care during Covid-19. Ambulatory practices and non-urgent surgeries were suspended to ensure staff and facilities avoid providers accessing capital and other sources of liquidity. Operations management focuses on health care systems focusing on improving the operational efficiency.

This project addresses a healthcare facility’s location, network, and planning capacity. Besides the combination of facilities and location-network design, the basic problem is optimally building a network that enables flows and satisfies some constraints that is beneficial to consumers and minimizes the financial strain on them in an already expensive field. The goal of this project is to identify which medical facility should service what zones, and what their respective specialty that should be open in to minimize costs and meet demands.

Nowadays, Health systems operations management is an area of health system engineering that focus on improving the operations and business of the health systems. Through Integer programming there many problems in Health systems that could be optimized such as appointment scheduling, staffing, health clinic location, EMS response optimization and rural health sample collection in this project we going to show how health clinic locations and capacity of appointment in each location that is open. Due to the rise of patients who are looking for health and humans are getting older; companies and care services have been minimized cost but trying to meet the most patient’s need. Thus, is critical to get a precise location depending on how much population and demand can include in the networks and for sure with high standards of quality and reliability.


**Data** 

Madison Healthcare network (MHN) provides many types of medical services across Wisconsin in the United States. The company is willing to provide an integrated health care network, the strategic goal is to present a plan to remain economically competitive and provide patient-centered care.

The company provided current location and capacity for each medical specialty. If the patient decides to make an appointment in another place outside of the MHN network the cost of the appointment is between 10-15% more than MHN appointments. If the distance between the facility and patient's location is too large, some patients choose another facility outside of the network.

To be more specific, the means and variability in appointment for each specialty in each region allows us to calculate demand for appointment in each region depending on medical specialty. The specialties are Cardiology, dental, dermatology, mental health, neurological, gynecology, Orthopedics, Pediatric and Pulmonary. Also, the data provides the cost of each specialty appointment and the cost for no-show that means the no presence of patients to the appointment.

Finally, the data provides distance between patient location and facilities, the location from L1 to L15 are defined through the zip code.


**Problem Description** 

The growth in the number of patients, the aging of population, increasing health care cost, technology shift and competition among service providers have affected health system.Considering plan patient- centered , the provider should provide high quality healthcare services at the minimum cost and reduce the distance between patients and their  network. However, capacity planning in health care system design could affect service quality. On the other hand, high capacity does not mean high demand. Thus, it is important to consider socioeconomic aspects, population, demand of services and costs.

This case in particular, the actual network is bringing consequences:
* When a hospital does not have enough capacity. Patients take a long time to receive care. This wait time leads to patients looking for an appointment in hospitals outside of the network. 
* The hospitals by lacking  point of view’s optimization, they may not use resources properly. Services that are not being used or lack of demand are increasing total cost.
* Poor capacity and demand  planning lead to a financial instability for no management of costs.
* The actual plan does not consider that there is a correlation between speciality appointments. For example patients that require neurology appointments there are 90% that they need mental health appointments.
* Finally being not able to provide care on time leads to patients leak providing poor outcomes.

**Methodology** 

The data originally was formatted to be on a multiple-page excel document, which did not translate well to being read in on a notebook, thus requiring some clean-up and manual formatting, which is done below. 

In [51]:
using JuMP, DataFrames, NamedArrays
##Data cleaning and formatting

specialty = ["Cardiology", 
             "Dental Care", 
             "Dermatology", 
             "Gastroenterology", 
             "Mental Health", 
             "Neurological", 
             "Obstetrics/Gynecology", 
             "Orthopedics", 
             "Pediatric", 
             "Pulmonary"]

cost = [425 150 120 375 125 200 350 400 125 375]

location = ["L1", "L2", "L3", "L4", "L5", "L6", "L7", "L8", "L9", "L10", "L11", "L12", "L13", "L14", "L15"]

facility = ["Flatlands", "Fisher", "Oceanside", "Greenmont"]


## distance of each facility from each location
dist = NamedArray(
[16 101 120 112
97.8 172 171 138
152 51.6 109 190
48.8 307 320 204
31.4 123 27.8 171
74.8 233 444 199
44.8 55.6 270 116
183 283 122 87.9
132 261 176 40.3
139 195 41.2 131
88.6 181 119 63.8
246 207 270 75.2
173 273 218 136
121 179 123 41.2
109 97.5 146 108], (location, facility), ("Location", "Facility"))

## maximum capacity of each facility
max_cap = [430 457 400 408]

## capacity of each facility in each specialty field
spec_cap = NamedArray(
[46 45 28 61
56 39 40 43
45 53 70 43
62 73 42 81
27 53 52 46
12 40 22 39
65 58 46 43
51 53 26 43
69 38 50 56
53 45 64 42], (specialty, facility), ("Specialty", "Facility"))

## demand of each location for the service of each specialty 
demand = NamedArray(
[127 219 121 135 138 218 118 216 162 86
206 110 225 250 165 171 229 148 229 130
116 182 99 203 106 95 213 152 194 191
239 98 122 135 149 173 227 233 152 147
167 81 145 190 84 161 83 217 218 153
201 113 145 157 175 137 131 186 179 200
42 126 161 249 204 155 200 252 200 207
186 97 164 176 219 187 102 92 162 152
201 124 63 135 187 116 173 172 216 158
176 118 146 158 109 171 180 155 184 177
153 167 180 243 145 230 43 234 148 189
126 123 94 169 154 215 181 146 99 246
132 177 195 120 116 213 123 181 181 145
97 177 192 135 109 128 106 221 203 164
127 191 101 233 138 125 199 145 111 214], (location, specialty), ("Location", "Specialty"))

cost_drive = 2;

Then, using a Cbc solver in an Integer Programming format, a model can be created, which seeks to minimize the cost of an appointment for a certain specialty at a certain facility in addition to the cost of travel, which would be the distance from a location to said facility times the cost of driving. This objective function would be subject to capacity constraints (both for each specialization and the overall facility capacity) as well as constraints for determining which location zones should be assigned to which facility and which facility should be specializing in what field such that demands are met. 

In [52]:
using JuMP, Cbc


spec_count = length(specialty)
loc_count = length(location)
fac_count = length(facility)


m = Model(Cbc.Optimizer)


@variable(m, serve[1:loc_count, 1:fac_count], Bin)


@variable(m, assign[1:spec_count, 1:fac_count], Bin)

@objective(m, Min, 
    sum(cost[i] * assign[i, j] for i = 1:spec_count, j = 1:fac_count) +
    sum(cost_drive * dist[location[i], facility[j]] * serve[i, j] for i = 1:loc_count, j = 1:fac_count))

@constraint(m, capacity_facility[j = 1:fac_count], sum(assign[i, j] * spec_cap[specialty[i], facility[j]] for i = 1:spec_count) <= max_cap[j])


@constraint(m, serve_location[i = 1:loc_count], sum(serve[i, j] for j = 1:fac_count) == 1)

@constraint(m, assign_specialty[i = 1:spec_count], sum(assign[i, j] for j = 1:fac_count) == 1)

optimize!(m)

serve_result = value.(serve)
assign_result = value.(assign)

for i = 1:location_count
    for j = 1:facility_count
        if serve_result[i, j] > 0.5
            println("Location ", location[i], " is served by the ", facility[j], " facility")
        end
    end
end

for i = 1:specialty_count
    for j = 1:facility_count
        if assign_result[i, j] > 0.5
            println(specialty[i], " should be specialized by the ", facility[j], " facility")
        end
    end
end
println("Optimal cost of medical service: ", objective_value(m))

Location L1 is served by the Flatlands facility
Location L2 is served by the Flatlands facility
Location L3 is served by the Fisher facility
Location L4 is served by the Flatlands facility
Location L5 is served by the Oceanside facility
Location L6 is served by the Flatlands facility
Location L7 is served by the Flatlands facility
Location L8 is served by the Greenmont facility
Location L9 is served by the Greenmont facility
Location L10 is served by the Oceanside facility
Location L11 is served by the Greenmont facility
Location L12 is served by the Greenmont facility
Location L13 is served by the Greenmont facility
Location L14 is served by the Greenmont facility
Location L15 is served by the Fisher facility
Cardiology should be specialized by the Oceanside facility
Dental Care should be specialized by the Oceanside facility
Dermatology should be specialized by the Fisher facility
Gastroenterology should be specialized by the Oceanside facility
Mental Health should be specialized by 

**Conclusion** 

Based on the above results, we see that each location now has an assigned facility, which is determined by minimizing travel cost and appointment cost while meeting demands for each specialty subject to capacity limits. This is informative in that for future projects, further optimization may be done to identify which locations need additional facilities or staffing, while addressing the current issue of increased medical needs and the consumer-side financial burden. The model also determines which facility should increase or begin specializing in a field of medical care, as it is most demanded of its assigned zones, which would assist medical facilities in determining how staffing should be distributed in a way that best fits its clientele's needs. 

To implement this model, we had to simplify the data because each factor makes harder the programming. There are many perspectives that this issue could be addressed. In this project we sought to redesign networks and the algorithm p- center to make sure all the patients could access an appointment, although moving forward, the model could be imporved to be more competitive and calculate how many appointments for each specialty we could afford and minimize the staff requirement to give a patient center service. But the truth is that more data and constraints the model could get overwhelmed. However, this is the first step to realize the many solutions that we can bring the most important example is how we can organized emergency systems and see how it could save lives.

Unfortunately, medical care in America is very financially draining, especially in recent times. However, by optimizing not for the profit margins on the business side, and rather on the consumer side, perhaps middle ground can be reached from both hospitals and the local populous where medical care can be readily available while not breaking the bank. 